https://docs.llamaindex.ai/en/stable/examples/vector_stores/FaissIndexDemo/

pip install llama-index-core llama-index-vector-stores-chroma chromadb     

pip install llama-index  

pip install ollama   

pip install llama-index-embeddings-ollama

In [1]:
from llama_index.core import Document
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline, IngestionCache
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,StorageContext
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.llms.ollama import Ollama
from llama_index.core.llms import ChatMessage  
from IPython.display import Markdown, display
import gradio as gr
from llama_index.core.prompts import PromptTemplate
from llama_index.core.query_engine import RetrieverQueryEngine

In [2]:
documentDir="/Users/anandpardeep/AI/docStore"
vectorDbLocation="/Users/anandpardeep/AI/vectorDB"
indexStorage="/Users/anandpardeep/AI/indexStorage"
system_message = "You are a helpful assistant"

template_str = """
You are an assistant providing security responses based on stored knowledge for password policies.

Context:
{context_str}

User Input:
{query_str}

Provide a relevant answer based on the context.
"""

prompt_template = PromptTemplate(template_str)

In [ ]:
documents = SimpleDirectoryReader(documentDir).load_data()
documents

In [4]:
embed_model = OllamaEmbedding(model_name="nomic-embed-text", base_url="http://localhost:11434")  # Ollama's embedding model

In [ ]:
chroma_client = chromadb.PersistentClient(path=vectorDbLocation)
vector_store = ChromaVectorStore(chroma_client=chroma_client, chroma_collection="rag_data")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model, vector_store=vector_store)
# Save the index
index.storage_context.persist(persist_dir=indexStorage)


In [6]:
llm = Ollama(model="llama3.2:latest", request_timeout=1200.0)

In [7]:
retriever = index.as_retriever()
query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm, text_qa_template=prompt_template,similarity_top_k=2)


pip install llama-index-core llama-index-vector-stores-chroma llama-index-embeddings-nomic chromadb


In [ ]:
def message_model(prompt):
  print(prompt)
  return query_engine.query(prompt).response


In [ ]:
# https://www.gradio.app/guides/quickstart
gr.Interface(fn=message_model, inputs=gr.Textbox(lines=2, label="Ask a question..."),
             outputs="textbox", flagging_mode="never").queue().launch(inbrowser=False,debug=False)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Whats the password policy
is abc12345 correct password
what is the pswrod policy
what is the recommended length of password
